In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
def get_month(string):
    return string.split("/")[0]

alc_df = pd.read_csv('alc_dataset.csv', nrows=3000000)
#alc_df = pd.read_csv('alc_dataset.csv')
alc_df["Month"] = alc_df["Date"].apply(get_month)
alc_df.drop(columns = ["Date", "Unnamed: 0"], inplace = True)
#alc_df = pd.get_dummies(alc_df)
alc_df
#alc_df.groupby(["Year", "Month", "County"]).sum()

FileNotFoundError: [Errno 2] No such file or directory: 'alc_dataset.csv'

In [ ]:
print(len(alc_df['Category Name'].unique()))
print(len(alc_df['Vendor Name'].unique()))
print(len(alc_df['County'].unique()))
print(len(alc_df['Store Name'].unique()))

In [ ]:
print(alc_df['Store Name'].unique())

In [ ]:
grouped_df = pd.get_dummies(alc_df, columns=['Store Name', 'Category Name', 'Vendor Name']).groupby(['Year', 'Month', 'County', 'Population']).sum().reset_index()
grouped_df

In [ ]:
grouped_df = pd.get_dummies(grouped_df)
grouped_df

In [ ]:
grouped_df['State Profit per capita'] = (grouped_df['State Bottle Retail'] - grouped_df['State Bottle Cost'])/grouped_df['Population']
grouped_df['Liters Sold per capita'] = grouped_df['Volume Sold (Liters)'] / grouped_df['Population']
grouped_df['Sales per capita (Dollars)'] = grouped_df['Sale (Dollars)'] / grouped_df['Population']
grouped_df['Bottles Sold per capita'] = grouped_df['Bottles Sold'] / grouped_df['Population']
#grouped_df.drop(columns=['State Bottle Retail', 'Volume Sold (Liters)', 'Sale (Dollars)', 'Bottles Sold'], inplace=True)
grouped_df

In [ ]:
def coef_mat(X, Y):
    xtx = np.linalg.inv(X.T.dot(X))
    beta_hat = xtx.dot(X.T.dot(Y))
    return beta_hat

def ridge_betas(X, Y, L):
    xtx = np.linalg.inv(X.T.dot(X) + np.identity(X.shape[1])*L)
    beta_hat = xtx.dot(X.T.dot(Y))
    return beta_hat

def ols_regression(X, Y):
    beta_hat = coef_mat(X, Y)
    return X.dot(beta_hat)

def ridge_regression(X, Y, L, beta_hat):
    return X.dot(beta_hat)

def lse(Y, y_hat):
    return (Y - y_hat).T.dot(Y - y_hat)



In [ ]:
# Bottles Sold per capita
scaler = StandardScaler()

Y = grouped_df['Bottles Sold per capita']
bottles_sold_df = grouped_df.drop(columns=['Bottles Sold per capita', 'Sales per capita (Dollars)', 'Liters Sold per capita', 'State Profit per capita', 'Bottles Sold'])
bottles_columns = pd.Series(bottles_sold_df.columns)
bottles_sold_df = scaler.fit_transform(bottles_sold_df)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bottles_sold_df, Y, test_size=0.2, random_state = 42)

In [ ]:
L = 100
betas1 = ridge_betas(X_train, y_train, L)

new_df = pd.concat([bottles_columns, pd.Series(betas1)], axis=1)
new_df.sort_values(by=1, ascending=False)
y_hat = ridge_regression(X_train, y_train, L, betas1)
err = lse(y_train, y_hat)
err

In [ ]:
L = 100
y_hat = ridge_regression(X_test, y_test, L, betas1)
err = lse(y_test, y_hat)
#print(y_test, y_hat)
err

In [ ]:
# Sales volume per capita
Y = grouped_df['Sales per capita (Dollars)']
sales_volume_df = grouped_df.drop(columns=['Bottles Sold per capita', 'Sales per capita (Dollars)', 'Liters Sold per capita', 'State Profit per capita', 'Sale (Dollars)'])
sales_volume_columns = pd.Series(sales_volume_df.columns)
sales_volume_df = scaler.fit_transform(sales_volume_df)
X_train, X_test, y_train, y_test = train_test_split(sales_volume_df, Y, test_size=0.2, random_state = 10)

# training set
L = 100
betas1 = ridge_betas(X_train, y_train, L)

new_df = pd.concat([sales_volume_columns, pd.Series(betas1)], axis=1)
new_df.sort_values(by=1, ascending=False)

In [ ]:
y_hat = ridge_regression(X_train, y_train, L, betas1)
err = lse(y_train, y_hat)
err

In [ ]:
L = 100
y_hat = ridge_regression(X_test, y_test, L, betas1)
err = lse(y_test, y_hat)
err

In [ ]:
# Volume of sold alcohol per capita
Y = grouped_df['Liters Sold per capita']
volume_df = grouped_df.drop(columns=['Bottles Sold per capita', 'Sales per capita (Dollars)', 'Liters Sold per capita', 'State Profit per capita', 'Volume Sold (Liters)'])
volume_columns = pd.Series(volume_df.columns)
volume_df = scaler.fit_transform(volume_df)
X_train, X_test, y_train, y_test = train_test_split(volume_df, Y, test_size=0.2, random_state = 10)

# training set
L = 100
betas1 = ridge_betas(X_train, y_train, L)

new_df = pd.concat([volume_columns, pd.Series(betas1)], axis=1)
new_df.sort_values(by=1, ascending=False)
y_hat = ridge_regression(X_train, y_train, L, betas1)
err = lse(y_train, y_hat)
err

In [ ]:
L = 100
y_hat = ridge_regression(X_test, y_test, L, betas1)
err = lse(y_test, y_hat)
err

In [ ]:
# State Profit per capita
Y = grouped_df['Sales per capita (Dollars)']
state_profit_df = grouped_df.drop(columns=['Bottles Sold per capita', 'Sales per capita (Dollars)', 'Liters Sold per capita', 'State Profit per capita', 'State Bottle Retail', 'State Bottle Cost'])
profit_columns = pd.Series(state_profit_df.columns)
state_profit_df = scaler.fit_transform(state_profit_df)
X_train, X_test, y_train, y_test = train_test_split(state_profit_df, Y, test_size=0.2, random_state = 10)

# training set
L = 100
betas1 = ridge_betas(X_train, y_train, L)

new_df = pd.concat([profit_columns, pd.Series(betas1)], axis=1)
new_df.sort_values(by=1, ascending=False)
y_hat = ridge_regression(X_train, y_train, L, betas1)
err = lse(y_train, y_hat)
err

In [ ]:
L = 100
y_hat = ridge_regression(X_test, y_test, L, betas1)
err = lse(y_test, y_hat)
err